# Model Comparison Framework

This notebook compares multiple classification algorithms for predicting prolonged ICU stays.

**Objectives:**
- Establish baseline performance across multiple classifiers
- Identify best-performing algorithms for this dataset
- Test SGDClassifier (required for partial fitting in federated learning)
- Compare results to Jim's logistic regression baseline (Accuracy: 70.71%, F1: 65.84%)

**Models to Test:**
1. Logistic Regression (baseline)
2. Random Forest
3. Gradient Boosting
4. SGDClassifier (supports partial_fit for federated learning)
5. Support Vector Machine (SVM)
6. Decision Tree
7. K-Nearest Neighbors (KNN)

## Setup and Imports

In [1]:
import sys
sys.path.append('../../src')

import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Classifiers
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Evaluation
from fedlearn.evaluation import evaluate_model
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
plt.style.use('default')
sns.set_palette('husl')

print("All imports successful!")

All imports successful!


## Load Data

Load the cleaned feature set from `v_features_icu_stay_clean` view.

In [2]:
# Connect to database
db_path = Path("../../data/duckdb/fedlearn.duckdb")
conn = duckdb.connect(str(db_path), read_only=True)

# Load data
df = conn.execute("SELECT * FROM v_features_icu_stay_clean").df()
conn.close()

print(f"Dataset shape: {df.shape}")
print(f"Target distribution:")
print(df['prolonged_stay'].value_counts())
print(f"\nClass balance: {df['prolonged_stay'].value_counts(normalize=True).round(4)}")

Dataset shape: (199646, 77)
Target distribution:
prolonged_stay
0    150206
1     49440
Name: count, dtype: int64

Class balance: prolonged_stay
0    0.7524
1    0.2476
Name: proportion, dtype: float64


## Prepare Features and Target

Using the same approach as Jim's baseline model.

In [3]:
# Target variable
y = df["prolonged_stay"]

# Features (drop ID, target, and high-cardinality text field)
X = df.drop(
    columns=["patientunitstayid", "los_days", "prolonged_stay", "apacheadmissiondx"]
)

# Define categorical and numeric features
categorical_features = [
    "gender",
    "ethnicity",
    "unittype",
    "unitadmitsource",
    "hospitaladmitsource",
    "admissiondx_category",
    "numbedscategory",
    "teachingstatus",
    "hospital_region",
]

numeric_features = [c for c in X.columns if c not in categorical_features]

print(f"Total features: {len(X.columns)}")
print(f"Numeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")

Total features: 73
Numeric features: 64
Categorical features: 9


## Train/Test Split

80/20 split with stratification to maintain class balance.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {len(X_train):,}")
print(f"Test set size: {len(X_test):,}")
print(f"\nTraining set class distribution:")
print(y_train.value_counts())
print(f"\nTest set class distribution:")
print(y_test.value_counts())

Training set size: 159,716
Test set size: 39,930

Training set class distribution:
prolonged_stay
0    120164
1     39552
Name: count, dtype: int64

Test set class distribution:
prolonged_stay
0    30042
1     9888
Name: count, dtype: int64


## Preprocessing Pipeline

Reusing Jim's preprocessing approach:
- **Numeric features**: Median imputation + StandardScaler
- **Categorical features**: Most frequent imputation + One-hot encoding

In [5]:
# Numeric preprocessing
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

# Categorical preprocessing
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

# Combined preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

print("Preprocessing pipeline configured successfully")

Preprocessing pipeline configured successfully


## Define Models to Compare

In [6]:
# Define all models to test
models = {
    "Logistic Regression": Pipeline(
        steps=[
            ("preprocess", preprocessor),
            (
                "clf",
                LogisticRegression(
                    max_iter=500,
                    n_jobs=-1,
                    class_weight="balanced",
                    solver="lbfgs",
                    random_state=42,
                ),
            ),
        ]
    ),
    "SGDClassifier": Pipeline(
        steps=[
            ("preprocess", preprocessor),
            (
                "clf",
                SGDClassifier(
                    loss="log_loss",  # For logistic regression
                    max_iter=1000,
                    tol=1e-3,
                    class_weight="balanced",
                    random_state=42,
                    n_jobs=-1,
                ),
            ),
        ]
    ),
    "Random Forest": Pipeline(
        steps=[
            ("preprocess", preprocessor),
            (
                "clf",
                RandomForestClassifier(
                    n_estimators=100,
                    max_depth=10,
                    min_samples_split=20,
                    min_samples_leaf=10,
                    class_weight="balanced",
                    random_state=42,
                    n_jobs=-1,
                ),
            ),
        ]
    ),
    "Gradient Boosting": Pipeline(
        steps=[
            ("preprocess", preprocessor),
            (
                "clf",
                GradientBoostingClassifier(
                    n_estimators=100,
                    learning_rate=0.1,
                    max_depth=5,
                    min_samples_split=20,
                    min_samples_leaf=10,
                    random_state=42,
                ),
            ),
        ]
    ),
    "Decision Tree": Pipeline(
        steps=[
            ("preprocess", preprocessor),
            (
                "clf",
                DecisionTreeClassifier(
                    max_depth=10,
                    min_samples_split=20,
                    min_samples_leaf=10,
                    class_weight="balanced",
                    random_state=42,
                ),
            ),
        ]
    ),
    "SVM (RBF)": Pipeline(
        steps=[
            ("preprocess", preprocessor),
            (
                "clf",
                SVC(
                    kernel="rbf",
                    C=1.0,
                    class_weight="balanced",
                    random_state=42,
                    probability=True,  # Enable probability estimates for ROC-AUC
                ),
            ),
        ]
    ),
    "K-Nearest Neighbors": Pipeline(
        steps=[
            ("preprocess", preprocessor),
            (
                "clf",
                KNeighborsClassifier(
                    n_neighbors=15,
                    weights="distance",
                    n_jobs=-1,
                ),
            ),
        ]
    ),
}

print(f"Configured {len(models)} models for comparison:")
for name in models.keys():
    print(f"  - {name}")

Configured 7 models for comparison:
  - Logistic Regression
  - SGDClassifier
  - Random Forest
  - Gradient Boosting
  - Decision Tree
  - SVM (RBF)
  - K-Nearest Neighbors


## Train and Evaluate All Models

This may take several minutes depending on your hardware.

In [ ]:
import time

results = {}
training_times = {}

print("Starting model training and evaluation...\n")
print("=" * 80)

for name, model in models.items():
    print(f"\nTraining {name}...")
    start_time = time.time()
    
    # Evaluate model (fit + predict + metrics)
    metrics = evaluate_model(name, model, X_train, y_train, X_test, y_test)
    
    training_time = time.time() - start_time
    training_times[name] = training_time
    
    # Calculate ROC-AUC (requires probability estimates)
    try:
        if hasattr(model.named_steps['clf'], 'predict_proba'):
            y_proba = model.predict_proba(X_test)[:, 1]
            roc_auc = roc_auc_score(y_test, y_proba)
            metrics['roc_auc'] = float(roc_auc)
            print(f"ROC-AUC   : {roc_auc:.4f}")
    except:
        metrics['roc_auc'] = None
    
    metrics['training_time'] = training_time
    results[name] = metrics
    
    print(f"Training time: {training_time:.2f}s")
    print("=" * 80)

print("\nAll models trained successfully!")

Starting model training and evaluation...


Training Logistic Regression...
Logistic Regression
----------------------------------------
Accuracy  : 0.7080
Precision : 0.6554
Recall    : 0.6960
F1-score  : 0.6602

ROC-AUC   : 0.7620
Training time: 6.93s

Training SGDClassifier...
SGDClassifier
----------------------------------------
Accuracy  : 0.7042
Precision : 0.6516
Recall    : 0.6913
F1-score  : 0.6559

ROC-AUC   : 0.7546
Training time: 4.73s

Training Random Forest...
Random Forest
----------------------------------------
Accuracy  : 0.7299
Precision : 0.6668
Recall    : 0.7014
F1-score  : 0.6751

ROC-AUC   : 0.7764
Training time: 6.76s

Training Gradient Boosting...
Gradient Boosting
----------------------------------------
Accuracy  : 0.7944
Precision : 0.7355
Recall    : 0.6486
F1-score  : 0.6686

ROC-AUC   : 0.7973
Training time: 170.55s

Training Decision Tree...
Decision Tree
----------------------------------------
Accuracy  : 0.6998
Precision : 0.6496
Recall    : 0.6900


## Results Summary

In [ ]:
# Create results dataframe
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('f1', ascending=False)

print("\n" + "=" * 100)
print("MODEL COMPARISON RESULTS")
print("=" * 100)
print(results_df.to_string())
print("\n" + "=" * 100)

# Highlight best model
best_model = results_df.index[0]
best_f1 = results_df.loc[best_model, 'f1']
print(f"\nBest Model (by F1-score): {best_model}")
print(f"F1-score: {best_f1:.4f}")
print(f"\nBaseline (Jim's LogReg): F1 = 0.6584")
print(f"Improvement: {(best_f1 - 0.6584) / 0.6584 * 100:.2f}%")

## Visualize Model Comparisons

In [ ]:
# Performance metrics comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1']
titles = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

for idx, (metric, title) in enumerate(zip(metrics_to_plot, titles)):
    ax = axes[idx // 2, idx % 2]
    
    # Sort by metric value
    sorted_results = results_df.sort_values(metric, ascending=True)
    
    # Plot
    y_pos = range(len(sorted_results))
    colors = ['green' if x > 0.7 else 'orange' if x > 0.6 else 'red' 
              for x in sorted_results[metric]]
    
    ax.barh(y_pos, sorted_results[metric], color=colors, edgecolor='black')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(sorted_results.index)
    ax.set_xlabel('Score')
    ax.set_title(f'{title} Comparison', fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    ax.set_xlim(0, 1)
    
    # Add baseline line (Jim's LogReg)
    if metric == 'accuracy':
        baseline = 0.7071
    elif metric == 'precision':
        baseline = 0.6536
    elif metric == 'recall':
        baseline = 0.6932
    elif metric == 'f1':
        baseline = 0.6584
    
    ax.axvline(baseline, color='blue', linestyle='--', linewidth=2, 
               label=f"Baseline: {baseline:.4f}")
    ax.legend()
    
    # Add value labels
    for i, val in enumerate(sorted_results[metric]):
        ax.text(val + 0.01, i, f'{val:.4f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Training time comparison
fig, ax = plt.subplots(figsize=(12, 6))

sorted_times = results_df.sort_values('training_time', ascending=True)
y_pos = range(len(sorted_times))

ax.barh(y_pos, sorted_times['training_time'], edgecolor='black', alpha=0.7)
ax.set_yticks(y_pos)
ax.set_yticklabels(sorted_times.index)
ax.set_xlabel('Training Time (seconds)')
ax.set_title('Training Time Comparison', fontweight='bold', fontsize=14)
ax.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, val in enumerate(sorted_times['training_time']):
    ax.text(val + 0.5, i, f'{val:.2f}s', va='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# ROC-AUC comparison (for models that support it)
roc_auc_results = results_df[results_df['roc_auc'].notna()].sort_values('roc_auc', ascending=True)

if len(roc_auc_results) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    y_pos = range(len(roc_auc_results))
    colors = ['green' if x > 0.75 else 'orange' if x > 0.7 else 'red' 
              for x in roc_auc_results['roc_auc']]
    
    ax.barh(y_pos, roc_auc_results['roc_auc'], color=colors, edgecolor='black')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(roc_auc_results.index)
    ax.set_xlabel('ROC-AUC Score')
    ax.set_title('ROC-AUC Comparison', fontweight='bold', fontsize=14)
    ax.grid(True, alpha=0.3, axis='x')
    ax.set_xlim(0, 1)
    
    # Add value labels
    for i, val in enumerate(roc_auc_results['roc_auc']):
        ax.text(val + 0.01, i, f'{val:.4f}', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("No models with ROC-AUC scores available")

## Confusion Matrix for Best Model

In [ ]:
# Get best model by F1-score
best_model_name = results_df.index[0]
best_model_pipeline = models[best_model_name]

# Get predictions
y_pred = best_model_pipeline.predict(X_test)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, cbar=True,
            xticklabels=['Not Prolonged (0)', 'Prolonged (1)'],
            yticklabels=['Not Prolonged (0)', 'Prolonged (1)'])
ax.set_xlabel('Predicted Label', fontweight='bold')
ax.set_ylabel('True Label', fontweight='bold')
ax.set_title(f'Confusion Matrix - {best_model_name}', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

# Print detailed metrics
print(f"\nConfusion Matrix for {best_model_name}:")
print(f"True Negatives (TN):  {cm[0, 0]:,}")
print(f"False Positives (FP): {cm[0, 1]:,}")
print(f"False Negatives (FN): {cm[1, 0]:,}")
print(f"True Positives (TP):  {cm[1, 1]:,}")
print(f"\nPer-class accuracy:")
print(f"  Class 0 (Not Prolonged): {cm[0, 0] / cm[0].sum():.4f}")
print(f"  Class 1 (Prolonged):     {cm[1, 1] / cm[1].sum():.4f}")

## Key Findings and Recommendations

In [ ]:
print("=" * 80)
print("KEY FINDINGS")
print("=" * 80)

print(f"\n1. BEST PERFORMING MODEL")
print(f"   Model: {best_model_name}")
print(f"   F1-Score: {results_df.loc[best_model_name, 'f1']:.4f}")
print(f"   Accuracy: {results_df.loc[best_model_name, 'accuracy']:.4f}")
print(f"   Training Time: {results_df.loc[best_model_name, 'training_time']:.2f}s")

print(f"\n2. SGDClassifier PERFORMANCE (for Partial Fitting)")
sgd_f1 = results_df.loc['SGDClassifier', 'f1']
print(f"   F1-Score: {sgd_f1:.4f}")
print(f"   Accuracy: {results_df.loc['SGDClassifier', 'accuracy']:.4f}")
print(f"   vs Best Model: {(sgd_f1 - results_df.loc[best_model_name, 'f1']):.4f} difference")
print(f"   Note: SGDClassifier supports partial_fit() for federated learning")

print(f"\n3. BASELINE COMPARISON (Jim's Logistic Regression)")
baseline_f1 = 0.6584
improvement = (best_f1 - baseline_f1) / baseline_f1 * 100
print(f"   Baseline F1: {baseline_f1:.4f}")
print(f"   Best F1: {best_f1:.4f}")
print(f"   Improvement: {improvement:+.2f}%")

print(f"\n4. FASTEST MODEL")
fastest_model = results_df.sort_values('training_time').index[0]
print(f"   Model: {fastest_model}")
print(f"   Training Time: {results_df.loc[fastest_model, 'training_time']:.2f}s")
print(f"   F1-Score: {results_df.loc[fastest_model, 'f1']:.4f}")

print("\n" + "=" * 80)
print("NEXT STEPS")
print("=" * 80)
print("")
print("1. Hyperparameter tuning for top-performing models")
print("2. Implement partial fitting with SGDClassifier to simulate federated learning")
print("3. Feature importance analysis for tree-based models")
print("4. Cross-validation for more robust performance estimates")
print("5. Test with different preprocessing strategies (e.g., different imputation methods)")
print("6. Address class imbalance with SMOTE or other sampling techniques")
print("")
print("=" * 80)

## Save Results for Future Reference

In [ ]:
# Save results to CSV
output_path = Path("../../results/model_comparison_results.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
results_df.to_csv(output_path)

print(f"Results saved to: {output_path}")
print(f"\nSummary:")
print(f"  - {len(models)} models evaluated")
print(f"  - Best F1-score: {best_f1:.4f} ({best_model_name})")
print(f"  - All results saved for future analysis")